In [1]:
import import_ipynb
import utility
import Model

In [2]:
from keras import backend as K
from utility import LoadDataNoDefCW
from Model import DFNet
import random
from keras.utils import np_utils
from keras.optimizers import Adamax
import numpy as np
import os

random.seed(0)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# Use only CPU
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

description = "Training and evaluating DF model for closed-world scenario on non-defended dataset"

print(description)
# Training the DF model
NB_EPOCH = 100   # Number of training epoch
print ("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 128 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 10000 # Packet sequence length
OPTIMIZER = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) # Optimizer

NB_CLASSES = 95 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

Training and evaluating DF model for closed-world scenario on non-defended dataset
Number of Epoch:  100


c:\ProgramData\Anaconda3\envs\nAIvis\lib\site-packages\keras\optimizers\optimizer_v2\adamax.py:99: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
# Data: shuffled and split between train and test sets
print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_valid, y_valid, X_test, y_test = LoadDataNoDefCW()
# Please refer to the dataset format in readme
K.set_image_data_format("channels_last") # tf is tensorflow

Loading and preparing data for training, and evaluating the model
Loading non-defended dataset for closed-world scenario


utility:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
utility:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Data dimensions:
X: Training data's shape :  (11400,)
y: Training data's shape :  (11400,)
X: Validation data's shape :  (3800,)
y: Validation data's shape :  (3800,)
X: Testing data's shape :  (3800,)
y: Testing data's shape :  (3800,)


utility:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [5]:
# Function to adjust the sequence length to a target length
def adjust_sequence_length(sequence, target_length=10000, padding_value=-1):
    if len(sequence) < target_length:
        # If the sequence is shorter than the target length, pad with the specified value
        sequence = sequence + [padding_value] * (target_length - len(sequence))
    else:
        # If the sequence is longer than the target length, truncate it
        sequence = sequence[:target_length]
    return sequence

# Adjust sequence lengths for each dataset
X_train = np.array([adjust_sequence_length(seq) for seq in X_train])
X_valid = np.array([adjust_sequence_length(seq) for seq in X_valid])
X_test = np.array([adjust_sequence_length(seq) for seq in X_test])

# Convert data to float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = np.array(y_train).astype('float32')
y_valid = np.array(y_valid).astype('float32')
y_test = np.array(y_test).astype('float32')


In [6]:
# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]

print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')
print(X_test.shape[0], 'test samples')

11400 train samples
3800 validation samples
3800 test samples


In [7]:
# Convert class vectors to categorical classes matrices
y_train = np_utils.to_categorical(y_train, NB_CLASSES)
y_valid = np_utils.to_categorical(y_valid, NB_CLASSES)
y_test = np_utils.to_categorical(y_test, NB_CLASSES)

In [8]:
# Building and training model
print ("Building and training DF model")

model = DFNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

Building and training DF model
Model compiled


In [9]:
# Start training
history = model.fit(X_train, y_train,
		batch_size=BATCH_SIZE, epochs=NB_EPOCH,
		verbose=VERBOSE, validation_data=(X_valid, y_valid))

Epoch 1/100
90/90 - 114s - loss: 4.3738 - accuracy: 0.0648 - val_loss: 4.8059 - val_accuracy: 0.0079 - 114s/epoch - 1s/step
Epoch 2/100
90/90 - 88s - loss: 3.6760 - accuracy: 0.1425 - val_loss: 5.6780 - val_accuracy: 0.0079 - 88s/epoch - 983ms/step
Epoch 3/100
90/90 - 88s - loss: 3.2534 - accuracy: 0.2211 - val_loss: 5.5144 - val_accuracy: 0.0095 - 88s/epoch - 983ms/step
Epoch 4/100
90/90 - 88s - loss: 2.9251 - accuracy: 0.2959 - val_loss: 5.4286 - val_accuracy: 0.0268 - 88s/epoch - 983ms/step
Epoch 5/100
90/90 - 88s - loss: 2.6756 - accuracy: 0.3497 - val_loss: 4.4772 - val_accuracy: 0.0861 - 88s/epoch - 983ms/step
Epoch 6/100
90/90 - 89s - loss: 2.4540 - accuracy: 0.4054 - val_loss: 3.3876 - val_accuracy: 0.1934 - 89s/epoch - 984ms/step
Epoch 7/100
90/90 - 89s - loss: 2.2605 - accuracy: 0.4510 - val_loss: 2.9110 - val_accuracy: 0.3055 - 89s/epoch - 984ms/step
Epoch 8/100
90/90 - 68s - loss: 2.0914 - accuracy: 0.4905 - val_loss: 2.3397 - val_accuracy: 0.4513 - 68s/epoch - 752ms/step
E

In [10]:
# Start evaluating model with testing data
score_test = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing accuracy:", score_test[1])

119/119 - 2s - loss: 0.3199 - accuracy: 0.9279 - 2s/epoch - 19ms/step
Testing accuracy: 0.9278947114944458
